In [1]:
# SISO 5G gNB-UE Simulation using AIRSTRAN D 2200

# Import or install Sionna
try:
    import sionna.rt
except ImportError as e:
    import os
    os.system("pip install sionna-rt")
    import sionna.rt

# Other imports
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import mitsuba as mi
import warnings

# Suppress warnings
warnings.filterwarnings("ignore", message="invalid value encountered in multiply")
warnings.filterwarnings("ignore", category=UserWarning, module="jupyter_client")

# Import relevant components from Sionna RT
from sionna.rt import load_scene, Transmitter, Receiver, Transmitter, Camera, PathSolver
from sionna.rt import AntennaArray, PlanarArray, SceneObject, ITURadioMaterial
from sionna.rt.antenna_pattern import antenna_pattern_registry

# Load empty scene
scene = load_scene("/home/samuel-rivera/Development/geo2sigmap/scenes/duke/scene.xml")

jitc_llvm_init(): LLVM API initialization failed ..


2025-12-15 01:26:54 WARN  [HDRFilm] Monochrome mode enabled, setting film output pixel format to 'luminance' (was rgb).


In [2]:
import sys
import os

# Add the src directory to the Python path
sys.path.append(os.path.abspath('../src'))

In [3]:
# Install plyfile if not available
try:
    from plyfile import PlyData
except ImportError:
    import os
    os.system("pip install plyfile")
    from plyfile import PlyData

In [4]:
# ============================================
# SISO Configuration: gNB to UE
# ============================================

# Set the operating frequency (n48 band for 5G)
scene.frequency = 3.7e9  # 3.7 GHz

# Define UE position (fixed to start)
ue_position = [10.0, 0.0, 0.0]   # UE position (x, y, z in meters)

# ============================================
# Antenna Configuration
# ============================================

# gNB antenna: 3GPP TR 38.901 pattern (AIRSTRAN D 2200)
gnb_pattern_factory = antenna_pattern_registry.get("tr38901")
gnb_pattern = gnb_pattern_factory(polarization="V")

# UE antenna: Isotropic pattern (typical for mobile devices)
ue_pattern_factory = antenna_pattern_registry.get("iso")
ue_pattern = ue_pattern_factory(polarization="V")

# SISO: Single antenna element at origin [0, 0, 0] for both TX and RX
single_element = np.array([[0.0, 0.0, 0.0]])  # Shape: (1, 3)

# Configure antenna arrays
scene.tx_array = AntennaArray(
    antenna_pattern=gnb_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

scene.rx_array = AntennaArray(
    antenna_pattern=ue_pattern,
    normalized_positions=single_element.T  # Shape: (3, 1)
)

# ============================================
# Add Receiver to Scene
# ============================================

# Create UE receiver
rx = Receiver(name="ue", position=ue_position, display_radius=0.03)
scene.add(rx)

# ============================================
# Configure Propagation Environment
# ============================================

# Disable scattering for basic simulation
for radio_material in scene.radio_materials.values():
    radio_material.scattering_coefficient = 0.0

In [5]:
from scene_parser import extract_building_info
from tx_placement import TxPlacement

In [6]:
# ============================================
# Place gNB on a Specific Building
# ============================================

scene_xml_path = "/home/samuel-rivera/Development/geo2sigmap/scenes/duke/scene.xml"
building_info = extract_building_info(scene_xml_path, verbose=True)

selected_building_id = 1  # Change this to your desired building number

# TxPlacement will create the transmitter if it doesn't exist and place it on the building
# Correct parameter order: (scene, tx_name, scene_xml_path, building_id, offset)
tx_placer = TxPlacement(scene, "gnb", scene_xml_path, selected_building_id, offset=40.0)
tx_placer.set_rooftop_center()

# Get reference to the transmitter (already added to scene by TxPlacement)
tx = tx_placer.tx
gnb_position = tx.position.numpy()

# Point antenna toward UE
tx.look_at(ue_position)

print(f"\nSuccess! gNB placed on building {selected_building_id}")
print(f"Position: {gnb_position}")

# ============================================
# Compute Propagation Paths
# ============================================

# Instantiate path solver
p_solver = PathSolver()

# Compute propagation paths
paths = p_solver(
    scene=scene,
    max_depth=5,
    max_num_paths_per_src=5000,
    los=True,
    specular_reflection=True,
    diffuse_reflection=False,
    refraction=False,
    synthetic_array=False,
    seed=41,
)

# ============================================
# Visualize Scene
# ============================================

# Setup camera
cam = Camera(position=(100.0, 100.0, 50.0))
cam.look_at(gnb_position)

# Preview the scene with propagation paths
scene.preview(
    paths=paths,
    resolution=[1000, 1000],
    clip_at=50,
    show_orientations=True
)

Available buildings in scene: 118 total

Building 1:
  Rooftop height: 14.00 m
  X range: [-331.33, -240.54] m (center: -285.94)
  Y range: [241.81, 322.53] m (center: 282.17)
  Vertices: 23

Building 2:
  Rooftop height: 14.00 m
  X range: [386.13, 504.99] m (center: 445.56)
  Y range: [472.21, 600.31] m (center: 536.26)
  Vertices: 11

Building 3:
  Rooftop height: 14.00 m
  X range: [-287.78, -200.50] m (center: -244.14)
  Y range: [46.28, 139.58] m (center: 92.93)
  Vertices: 6

Building 4:
  Rooftop height: 17.50 m
  X range: [-35.79, 70.65] m (center: 17.43)
  Y range: [447.01, 523.88] m (center: 485.45)
  Vertices: 28

Building 5:
  Rooftop height: 17.50 m
  X range: [-67.31, -6.57] m (center: -36.94)
  Y range: [23.19, 87.26] m (center: 55.23)
  Vertices: 17

Building 6:
  Rooftop height: 17.50 m
  X range: [260.77, 340.01] m (center: 300.39)
  Y range: [387.33, 468.23] m (center: 427.78)
  Vertices: 14

Building 7:
  Rooftop height: 14.00 m
  X range: [160.20, 282.47] m (cente

In [ ]:
from boresight_pathsolver import (
    optimize_boresight_pathsolver,
    create_target_radiomap
)
import matplotlib.pyplot as plt

# Map configuration (same as before)
map_config = {
    'center': [float(gnb_position[0]), float(gnb_position[1]), 0.0],
    'size': [1000, 1000],
    'cell_size': (10, 10),
    'ground_height': 0.0,
}

angle_map = [{'angle_start': 0, 'angle_end': 120, 'power_dbm': -70, 'relative_power': 'high'},
            {'angle_start': 120, 'angle_end': 360, 'power_dbm': -100, 'relative_power': 'low'}]

# CHANGE: Use path_loss_sector instead of angular_sectors
target_map = create_target_radiomap(
    map_config,
    target_type='angular_sectors',
    angular_sectors=angle_map,
    tx_position=gnb_position,
    auto_scale_power=True
    #sector_angle=45,        
    #sector_width=120,      
    #frequency_GHz=3.5,
    #path_loss_exponent=2.5
)

# Calculate the "center" look-at position. The look_at position should be level with the BS height.
center_x = map_config['center'][0] + 100.0
center_y = map_config['center'][1] + 100.0
center_z = float(gnb_position[2])

# Rest is the same
best_boresight, loss_hist, bore_hist, grad_hist = optimize_boresight_pathsolver(
    scene=scene,
    tx_name="gnb",
    map_config=map_config,
    scene_xml_path=scene_xml_path,
    target_map=target_map,
    initial_boresight=[center_x, center_y, center_z],
    num_sample_points=300,
    learning_rate=.05,      
    num_iterations=100,     
    loss_type='huber',
    verbose=True,
    seed=42,
    # Enable frame saving
    save_radiomap_frames=True,
    frame_save_interval=25,
    output_dir="home/tingjunlab/Development/optimize_tx/optimization_gif"
)

print(f"Optimized boresight: ({best_boresight[0]:.1f}, {best_boresight[1]:.1f})")

/tmp/ipykernel_18293/1403609360.py:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'center': [float(gnb_position[0]), float(gnb_position[1]), 0.0],
/tmp/ipykernel_18293/1403609360.py:34: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  center_z = float(gnb_position[2])



[Auto-Scale Power] Estimated achievable levels:
  High:     -100.0 dBm (main lobe, near)
  Medium:   -122.3 dBm (main lobe, far)
  Low:      -140.3 dBm (sidelobe, near)
  Very Low: -140.3 dBm (sidelobe, far)
  Center distance: 54.0 m
  Corner distance: 709.2 m


Boresight Optimization using PathSolver + AD
Initial boresight: (-185.9, 382.2, 54.0)
Learning rate: 0.05
Iterations: 100
Sample points: 600
Loss type: huber
Map config: {'center': [-285.93841552734375, 282.16790771484375, 0.0], 'size': [1000, 1000], 'cell_size': (10, 10), 'ground_height': 0.0}

TX placement mode: skip (using current position)
  Current TX position: (-285.94, 282.17, 54.00)
TX height: 54.0m
Boresight Z constraint: must be < 54.0m (no pointing upward)

Target values range: [-140.3, -100.0] dB
Removing 1 existing receiver(s): ['ue']
Learning rate scheduler enabled (patience=20, threshold=0.05%)
RadioMap frames will be saved to: home/tingjunlab/Development/optimize_tx/optimization_gif
Saving every 25 iteration(s)

KeyboardInterrupt: 

In [ ]:
from boresight_pathsolver import create_optimization_gif

# Define sector overlays to visualize coverage zones
sector_overlays = [
    {
        'angle_start': 0,
        'angle_end': 120,
        'color': 'green',     # High coverage sector
        'alpha': 0.25
    },
    {
        'angle_start': 120,
        'angle_end': 360,
        'color': 'red',       # Low coverage sector
        'alpha': 0.15
    }
]

# Create GIF from saved frames with sector overlay
create_optimization_gif(
    frame_dir="home/tingjunlab/Development/optimize_tx/optimization_gif",
    output_path="boresight_optimization.gif",
    duration=300,  # 300ms per frame
    loop=0,  # Loop forever
    sector_angles=sector_overlays,
    tx_position=gnb_position,
    map_config=map_config
)

In [ ]:
from sionna.rt import RadioMapSolver
from diagnostic_utils import analyze_coverage, print_analysis, visualize_results
import diagnostic_utils, optimizer_diagnostics
from optimizer_diagnostics import diagnose_optimization, print_diagnostics, plot_diagnostics

# Apply the optimized boresight
#m_best_boresight = mi.Point3f(best_mse[0], best_mse[1], best_mse[2])
tx.look_at(mi.Point3f(float(best_boresight[0]), float(best_boresight[1]), float(best_boresight[2])))
#stx.position = mi.Point3f(float(best_tx[0]), float(best_tx[1]), float(building["z_height"] + 20.0))

print("=" * 70)
print("RADIO MAP VISUALIZATION")
print("=" * 70)
print(f"Optimized boresight: ({best_boresight[0]:.1f}, {best_boresight[1]:.1f}, {best_boresight[2]:.1f})")
print(f"TX location: {tx.position}")
print(f"Coverage center: {map_config['center']}")
print()

# Generate radio map EXACTLY matching the optimization configuration
rm_solver = RadioMapSolver()
rm = rm_solver(
    scene,
    max_depth=5,
    samples_per_tx=int(6e8),
    cell_size=map_config['cell_size'],  # ✓ USE map_config
    center=map_config['center'],        # ✓ USE map_config
    orientation=[0, 0, 0],
    size=map_config['size'],            # ✓ USE map_config
    los=True,
    specular_reflection=True,
    diffuse_reflection=True,
    refraction=False,
    stop_threshold=None,
)

print("Radio map configuration:")
print(f"  Center: {map_config['center']}")
print(f"  Size: {map_config['size']}")
print(f"  Cell size: {map_config['cell_size']}")
print()

# Extract signal levels
rss_watts = rm.rss.numpy()[0, :, :]
signal_strength_dBm = 10.0 * np.log10(rss_watts + 1e-30) + 30.0

# Automatic analysis based on target type and loss function
analysis = analyze_coverage(
    signal_strength_dBm,
    target_map,
    loss_type='mse',  # Change to 'cross_entropy' when using that loss
    map_config=map_config
)

# Print results
#success = print_analysis(analysis)

# Visualize
visualize_results(signal_strength_dBm, target_map, analysis)

# Diagnostics to adjust the optimization
diag = diagnose_optimization(loss_hist, bore_hist, grad_hist)
print_diagnostics(diag)
plot_diagnostics(loss_hist, bore_hist, grad_hist)

# Scene preview
scene.preview(radio_map=rm);